# A notebook to try out different llm promts and models until the setup is complete. 

For the diffrent Prompts i will orient myself on the following simular Projects: 

* https://arxiv.org/pdf/2403.11996
* https://towardsdatascience.com/how-to-convert-any-text-into-a-graph-of-concepts-110844f22a1a
* https://towardsdatascience.com/text-to-knowledge-graph-made-easy-with-graph-maker-f3f890c0dbe8
